In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from scipy.sparse.csgraph import laplacian

pp = dict(surface='white', hemi='both')

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    9.0s remaining:    9.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    9.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    9.5s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished


In [3]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=4, extents=(1, 40), duration_of_trial=1, target_snr=99999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
y = evoked.data
x = stc.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:02<00:00,  1.05s/it]


Using pyvistaqt 3d backend.

Using control points [0.00000000e+00 0.00000000e+00 4.75874068e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Algo

## Helpers

In [4]:
def generator(fwd, batch_size=1284, n_sources=2, n_orders=2, verbose=0):

    # Parameters
    # batch_size = 1111
    # n_sources=5
    # n_orders=2
    # verbose=0

    amplitude_range = (0.001,1)
    adjacency = mne.spatial_src_adjacency(fwd["src"], verbose=verbose).toarray()
    gradient = abs(laplacian(adjacency))
    leadfield = fwd["sol"]["data"]
    leadfield -= leadfield.mean()

    leadfield_norm = deepcopy(leadfield)
    leadfield_norm /= np.linalg.norm(leadfield_norm, axis=0)

    n_chans, n_dipoles = leadfield.shape

    sources = np.identity(n_dipoles)
    for _ in range(n_orders-1):
        new_sources = sources[-n_dipoles:, -n_dipoles:] @ gradient
        new_sources /= new_sources.max(axis=0)
        sources = np.concatenate( [sources, new_sources], axis=0 )



    n_candidates = sources.shape[0]

    while True:
        print("ye")
        # select sources and their amplitudes
        n_sources_batch = np.random.randint(1, n_sources+1, batch_size)
        selection = [np.random.randint(0, n_candidates, n) for n in n_sources_batch]
        amplitudes = [np.random.uniform(*amplitude_range, n) * np.random.choice([1,-1], n) for n in n_sources_batch]
        # print(amplitudes[0], type(amplitudes[0]), sources[selection[0]].shape)
        y = np.stack([(amplitudes[i] @ sources[selection[i]]) / len(amplitudes[i]) for i in range(batch_size)], axis=0)
        y = (y.T / abs(y).max(axis=1)).T
        x = leadfield @ y.T
        # scale x
        x -= x.mean(axis=0)
        x /= np.linalg.norm(x, axis=0)
        
        # OMP
        # x = leadfield_norm.T @ x
        # x /= np.linalg.norm(x, axis=0)

        y = y[:, np.newaxis, :]
        x = x.T[:, np.newaxis, :]
        # y = (y!=0).astype(float)
        
        yield (x, y)

        

In [5]:
gen = generator(fwd, batch_size=5, n_sources=n_sources)
x_test, y_test = gen.__next__()


NameError: name 'n_sources' is not defined

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Lambda, Activation
from tensorflow.keras import backend as K
tf.keras.backend.set_image_data_format('channels_last')

# L1 = lambda x: K.mean(K.abs(x))
# LR = lambda x: K.mean(K.abs(x)) / K.mean(K.square(x))
# LR = lambda x: K.mean(K.square(x)) / K.mean(K.abs(x))

n_channels = evoked.data.shape[0]
n_dipoles = x.shape[0]

n_dense_units = 300
activation_function = "relu"
batch_size = 1284
n_sources = 5
n_orders = 2
epochs = 100
n_hl = 1
eps = 0.1

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')

fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function,
            name='FC1'))(inputs)

for n in range(n_hl):
    fc1 = TimeDistributed(Dense(n_dense_units, 
                activation=activation_function,
                name='FC'+str(n+2)))(fc1)

out = TimeDistributed(Dense(n_dipoles, 
            activation="linear", 
            name='Output'))(fc1)

model = tf.keras.Model(inputs=inputs, outputs=out, name='AutoANN')
model.compile(loss="cosine_similarity", optimizer="adam")
model.summary()

model.fit(x=generator(fwd, batch_size=batch_size, n_sources=n_sources, n_orders=n_orders), epochs=epochs,  steps_per_epoch=20)



Model: "AutoANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, None, 64)]        0         
                                                                 
 time_distributed (TimeDistr  (None, None, 300)        19500     
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 300)        90300     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 1284)       386484    
 tributed)                                                       
                                                                 
Total params: 496,284
Trainable params: 496,284
Non-trainable params: 0
_____________________________________________________

In [7]:
gen = generator(fwd, batch_size=5, n_sources=n_sources, n_orders=n_orders)
x_test, y_test = gen.__next__()

y_hat = model.predict(x_test)

stc_ = stc.copy()
stc_.data = y_test[:, 0].T
stc_.plot(**pp, brain_kwargs=dict(title="Ground Truths"))

stc_ = stc.copy()
stc_.data = y_hat[:, 0].T
stc_.plot(**pp, brain_kwargs=dict(title="Preds"))

ye
1/1 [==============================] - 0s 91ms/step
Using control points [0. 0. 1.]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [0.02136925 0.02453144 0.05638586]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [0.01494075 0.01618905 0.02966308]
Using control points [0.01494075 0.01618905 0.02966308]
